## Question - 1

In [ ]:
from confluent_kafka import Producer


conf = {'bootstrap.servers':'pkc-n3603.us-central1.gcp.confluent.cloud:9092',
        'sasl.username':'XXXXXXXXXX',
         'sasl.password':'XXXXXXXXX',
         'sasl.mechanism':'PLAIN',
         'security.protocol':'SASL_SSL'
}


topic = 'topic_0'
message  = "Hello Uday"

producer = Producer(conf)

def delivery_report(err, msg):
    if err is not None:
        print(f'Message delivered failed: {err}')
    else:
        print(f'Message delivered to {msg.topic()} [{msg.partition()}]')

producer.produce(topic, message.encode('utf-8'), callback = delivery_report)
                
producer.flush()


## Question - 2

In [ ]:
from confluent_kafka import Consumer


conf = {
    'bootstrap.servers': 'pkc-n3603.us-central1.gcp.confluent.cloud:9092',
    'sasl.username':'XXXXXXXXXX',
    'sasl.password':'XXXXXXXXX',
    'sasl.mechanism': 'PLAIN',
    'security.protocol': 'SASL_SSL',
    'group.id': 'my_consumer_group',
    'auto.offset.reset': 'earliest',
}

topic = 'topic_0'

consumer = Consumer(conf)

# Subscribe to the Kafka topic
consumer.subscribe([topic])

# Continuously consume messages
while True:
    message = consumer.poll(1.0)

    if message is None:
        continue

    if message.error():
        print(f"Error: {message.error()}")
        continue

    print(f"Received message: {message.value().decode('utf-8')}")

consumer.close()


## Question - 3

In [ ]:
from confluent_kafka.admin import AdminClient, NewTopic


conf = {
    'bootstrap.servers': 'pkc-n3603.us-central1.gcp.confluent.cloud:9092',
    'sasl.username':'XXXXXXXXXX',
    'sasl.password':'XXXXXXXXX',
    'sasl.mechanism': 'PLAIN',
    'security.protocol': 'SASL_SSL'
}

admin_client = AdminClient(conf)

# a) Create a new Kafka topic
new_topic = NewTopic("topic_1", num_partitions=3, replication_factor=1)
admin_client.create_topics([new_topic],validate_only = False)

# b) List existing Kafka topics 
topics = admin_client.list_topics().topics
print("Existing topics:")
for topic in topics:
    print(topic)

# c) Delete an existing Kafka topic
admin_client.delete_topics(["my_topic"])

# Verify deletion by listing topics again
topics = admin_client.list_topics().topics
print("Updated topic list:")
for topic in topics:
    print(topic)


## Question - 4

In [ ]:
from confluent_kafka import Producer, Consumer, KafkaError

conf = {
    'bootstrap.servers': 'pkc-n3603.us-central1.gcp.confluent.cloud:9092',
    'sasl.username': 'XXXXXXXXXX',
    'sasl.password': 'XXXXXXXXX',
    'sasl.mechanism': 'PLAIN',
    'security.protocol': 'SASL_SSL'
}

topic = 'my_topic'
# Delivery Report
def delivery_report(err, msg):
    if err is not None:
        print(f'Message delivered failed: {err}')
    else:
        print(f'Message delivered to {msg.topic()} [{msg.partition()}]')

# Producer code
def produce_messages():
    producer = Producer(conf)

    # Produce messages to the Kafka topic
    for i in range(10):
        message = f"Message {i}"
        producer.produce(topic, value=message, callback = delivery_report)

    # Flush and close the producer
    producer.flush()
    

# Consumer code
def consume_messages():
    conf['group.id'] = 'my_consumer_group'
    conf['auto.offset.reset'] = 'earliest'
    consumer = Consumer(conf)

    # Subscribe to the Kafka topic
    consumer.subscribe([topic])

    # Continuously consume messages
    while True:
        message = consumer.poll(1.0)

        if message is None:
            continue
        if message.error():
            if message.error().code() == KafkaError._PARTITION_EOF:
                continue
            else:
                print(f"Error: {message.error()}")
                break

        # Process the received message
        print(f"Received message: {message.value().decode('utf-8')}")
 
        # Manually commit the offset after processing the message
        consumer.commit(message)

    # Close the consumer
    consumer.close()

# Test the end-to-end flow
produce_messages()
consume_messages()


## Question - 5

IF 2 or More Consumers consume from a topic from same consumer Group Kafka Can Automatically Manages and 
maintains that no 2 consumers from same consumer group consume same partition at a time

In [10]:
# Producer Code

from confluent_kafka import Producer, Consumer, KafkaError

conf = {
    'bootstrap.servers': 'pkc-n3603.us-central1.gcp.confluent.cloud:9092',
    'sasl.username': 'XXXXXXXXXX',
    'sasl.password': 'XXXXXXXXX',
    'sasl.mechanism': 'PLAIN',
    'security.protocol': 'SASL_SSL'
}

topic = 'my_topic'
# Delivery Report
def delivery_report(err, msg):
    if err is not None:
        print(f'Message delivered failed: {err}')
    else:
        print(f'Message delivered to {msg.topic()} [{msg.partition()}]')

# Producer code
def produce_messages():
    producer = Producer(conf)

    # Produce messages to the Kafka topic
    for i in range(10):
        message = f"Message {i}"
        producer.produce(topic, value=message, callback = delivery_report)

    # Flush and close the producer
    producer.flush()

# Producer call
produce_messages()


Message delivered to my_topic [1]

Message delivered to my_topic [1]

Message delivered to my_topic [0]

Message delivered to my_topic [0]

Message delivered to my_topic [0]

Message delivered to my_topic [2]

Message delivered to my_topic [2]

Message delivered to my_topic [2]

Message delivered to my_topic [2]

Message delivered to my_topic [2]

In [ ]:
# Consumer Code

from confluent_kafka import Consumer, KafkaError

conf = {
    'bootstrap.servers': 'pkc-n3603.us-central1.gcp.confluent.cloud:9092',
    'sasl.username': 'XXXXXXXXXX',
    'sasl.password': 'XXXXXXXXXX',
    'sasl.mechanism': 'PLAIN',
    'security.protocol': 'SASL_SSL',
    'group.id': 'my_consumer_group',
    'auto.offset.reset': 'earliest',
}

topic = 'my_topic'

def consume_messages(consumer_id):
    consumer = Consumer(conf)

    # Subscribe to the Kafka topic
    consumer.subscribe([topic])

    # Continuously consume messages
    while True:
        message = consumer.poll(1.0)

        if message is None:
            continue

        if message.error():
            if message.error().code() == KafkaError._PARTITION_EOF:
                continue
            else:
                print(f"Error: {message.error()}")
                break

        # Process the received message
        print(f"Consumer {consumer_id}: Received message: {message.value().decode('utf-8')} from Partition {message.partition()}")

        # Manually commit the offset after processing the message
        consumer.commit(message)

In [ ]:
# Consumer - 1 from Consumer Group - my_consumer_group

consume_messages("Consumer_1")

Consumer Consumer_1: Received message: Message 3 from Partition 0

Consumer Consumer_1: Received message: Message 6 from Partition 0

Consumer Consumer_1: Received message: Message 9 from Partition 0

Consumer Consumer_1: Received message: Message 1 from Partition 1

Consumer Consumer_1: Received message: Message 2 from Partition 1

In [ ]:
# Consumer - 2 from Consumer Group - my_consumer_group

consume_messages("Consumer_2")

Consumer Consumer_2: Received message: Message 0 from Partition 2

Consumer Consumer_2: Received message: Message 4 from Partition 2

Consumer Consumer_2: Received message: Message 5 from Partition 2

Consumer Consumer_2: Received message: Message 7 from Partition 2

Consumer Consumer_2: Received message: Message 8 from Partition 2

You Can check the difference between the 2 consumers that consumer 1 got 1,2,3,6,9 messages and consumer 2 got 0,4,5,7,8 messages at a time